In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import datetime as datetime
from tqdm import tqdm
import networkx as nx
import random

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from src.utils.CreateFeatures import CreateFeatures
from src.pygcn.SiameseGNN import SiameseGNN
from src.pygcn.GraphSAGE import SiameseGNN_SAGE
from src.pygcn.graph_isomorphism import SiameseGNN_GIN
from torch_geometric.data import DataLoader
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch
import torch_geometric.data as data
from src.utils.sample import sample_pairs
from src.utils.misc import collate
import itertools

ImportError: cannot import name 'to_batch' from 'torch_geometric.utils' (/Users/sinclaireschuetze/Documents/GitHub/Trade-GNN-ChangePoint/.env/lib/python3.11/site-packages/torch_geometric/utils/__init__.py)

## All Events

In [ ]:
years = range(1962,2019)

In [ ]:
all_nodes = ['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM',
       'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD',
       'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA',
       'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN',
       'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB',
       'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DMA', 'DNK', 'DOM', 'DZA',
       'ECU', 'EGY', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FSM',
       'GAB', 'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC',
       'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI',
       'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA',
       'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KNA', 'KOR',
       'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO', 'LTU',
       'LUX', 'LVA', 'MAC', 'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL',
       'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MNP', 'MOZ', 'MRT',
       'MUS', 'MWI', 'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR',
       'NPL', 'NRU', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW',
       'PNG', 'POL', 'PRT', 'PRY', 'PSE', 'PYF', 'QAT', 'ROU', 'RUS',
       'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SMR',
       'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 'SXM',
       'SYC', 'SYR', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON',
       'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UGA', 'UKR', 'URY', 'USA',
       'UZB', 'VCT', 'VEN', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB',
       'ZWE']

In [4]:
with open("src/pygcn/all_graphs.pkl", "rb") as f:         
    all_graphs = pkl.load(f)

In [5]:
all_loader = DataLoader(all_graphs, batch_size=32)

/Users/sinclaireschuetze/Documents/GitHub/Trade-GNN-ChangePoint/.env/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## sGNN with GCN Encoder and 3 Features

In [6]:
def check_crisis_years(year_pairs, crisis_years):
    result = []
    for pair in year_pairs:
        start, end = pair
        # Check if any crisis year is between the pair or equals the later year
        if any(start < year <= end for year in crisis_years):
            result.append(0)
        else:
            result.append(1)
    return result

In [7]:
def get_year_pairs(year_range):
    return [(year1, year2) for year1 in year_range for year2 in year_range if year2 > year1]

def get_loader_pairs(dataset):
    return [(dataset[i], dataset[j]) for i in range(len(dataset)) for j in range(len(dataset)) if j > i]

In [8]:
crisis_years = [1962, 1967, 1973, 1978, 1981, 1989, 1993, 1996, 2002, 2007, 2012, 2014, 2016]
all_pairs = get_year_pairs(years)
all_y = check_crisis_years(all_pairs, crisis_years)
all_loader_pairs = get_loader_pairs(all_loader.dataset)

In [9]:
labeled_pairs_all = list(zip(all_loader_pairs, all_y))
flattened_all = [(a, b, c) for ((a, b), c) in labeled_pairs_all]

In [10]:
from sklearn.model_selection import train_test_split
flattened_train, flattened_test = train_test_split(flattened_all, test_size=0.40, random_state=42)
flattened_test, flattened_val = train_test_split(flattened_test, test_size=0.5, random_state=42)

In [11]:
import random
positive_samples = [item for item in flattened_train if item[2] == 1]
negative_samples = [item for item in flattened_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = positive_samples * (diff // len(positive_samples)) + random.sample(positive_samples, diff % len(positive_samples))
    balanced_data = negative_samples + positive_samples + positive_samples_upsampled
else:
    balanced_data = flattened_train

random.shuffle(balanced_data)

In [12]:
from sklearn.metrics import f1_score
def run_model(model, train_data, val_data):
    torch.manual_seed(42)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Adjust step_size and gamma as needed
    criterion = nn.BCELoss()

    for epoch in tqdm(range(10)):
        model.train()
        train_losses = []
        for data1, data2, label in train_data:

            optimizer.zero_grad()
            out = model(data1, data2)
            label = torch.tensor(label).view(1).float()
            loss = criterion(out.squeeze(0), label)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        scheduler.step()  # Add this line to update the learning rate

        model.eval()
        with torch.no_grad():
            val_losses = []

            val_pred = []
            val_truth = []

            correct = 0
            total = 0
            for data1, data2, label in val_data:
                out = model(data1, data2)
                label = torch.tensor(label).view(1).float()
                val_loss = criterion(out.squeeze(0), label)
                val_losses.append(val_loss.item())

                predictions = torch.round(out.squeeze())

                val_pred.append(predictions)
                val_truth.append(label)

                correct += (predictions == label).sum().item()
                total += 1

            val_loss = sum(val_losses) / len(val_losses)
            val_accuracy = correct / total

        print(f'Epoch: {epoch+1}, Training Loss: {sum(train_losses)/len(train_losses)}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}, Validation F1 Score: {f1_score(val_truth, val_pred)}')

In [13]:
model = SiameseGNN(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

 10%|█         | 1/10 [00:37<05:35, 37.31s/it]

Epoch: 1, Training Loss: 0.4723623523549062, Validation Loss: 0.5961926792631858, Validation Accuracy: 0.69375, Validation F1 Score: 0.3466666666666667


 20%|██        | 2/10 [01:18<05:17, 39.69s/it]

Epoch: 2, Training Loss: 0.3087776175932438, Validation Loss: 0.40030294487369245, Validation Accuracy: 0.81875, Validation F1 Score: 0.4727272727272727


 30%|███       | 3/10 [01:58<04:36, 39.55s/it]

...

Epoch: 8, Training Loss: 0.15447523622658735, Validation Loss: 0.31328924737917985, Validation Accuracy: 0.871875, Validation F1 Score: 0.5176470588235293


 90%|█████████ | 9/10 [05:54<00:39, 39.31s/it]

Epoch: 9, Training Loss: 0.1529877919290799, Validation Loss: 0.19641892725885554, Validation Accuracy: 0.9125, Validation F1 Score: 0.588235294117647


100%|██████████| 10/10 [06:33<00:00, 39.37s/it]

Epoch: 10, Training Loss: 0.13028820348460535, Validation Loss: 0.19641803601971333, Validation Accuracy: 0.9125, Validation F1 Score: 0.5483870967741936


In [18]:
model = SiameseGNN_SAGE(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

 10%|█         | 1/10 [00:30<04:32, 30.22s/it]

Epoch: 1, Training Loss: 0.4515431257114814, Validation Loss: 0.4499057385895867, Validation Accuracy: 0.796875, Validation F1 Score: 0.4347826086956523


 20%|██        | 2/10 [00:59<03:57, 29.68s/it]

Epoch: 2, Training Loss: 0.25942882886562724, Validation Loss: 0.27011743815382944, Validation Accuracy: 0.88125, Validation F1 Score: 0.5365853658536586


 30%|███       | 3/10 [01:28<03:26, 29.48s/it]

...

Epoch: 8, Training Loss: 0.09914862341420039, Validation Loss: 0.1517112492248998, Validation Accuracy: 0.94375, Validation F1 Score: 0.7096774193548386


 90%|█████████ | 9/10 [04:27<00:29, 29.97s/it]

Epoch: 9, Training Loss: 0.08385693140068014, Validation Loss: 0.23891917914224906, Validation Accuracy: 0.91875, Validation F1 Score: 0.6486486486486487


100%|██████████| 10/10 [04:56<00:00, 29.63s/it]

Epoch: 10, Training Loss: 0.07029848945007026, Validation Loss: 0.16799969493768002, Validation Accuracy: 0.95, Validation F1 Score: 0.7419354838709677


In [19]:
model = SiameseGNN_GIN(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

 10%|█         | 1/10 [00:29<04:28, 29.87s/it]

Epoch: 1, Training Loss: 0.5278637860370362, Validation Loss: 0.5778978556394577, Validation Accuracy: 0.634375, Validation F1 Score: 0.3157894736842105


 20%|██        | 2/10 [00:59<03:59, 29.97s/it]

Epoch: 2, Training Loss: 0.3352356116408253, Validation Loss: 0.33384038505901115, Validation Accuracy: 0.7625, Validation F1 Score: 0.3870967741935484


 30%|███       | 3/10 [01:30<03:30, 30.04s/it]

...

Epoch: 8, Training Loss: 0.16156419942808395, Validation Loss: 0.2556030187530268, Validation Accuracy: 0.909375, Validation F1 Score: 0.6419753086419752


 90%|█████████ | 9/10 [04:36<00:30, 30.88s/it]

Epoch: 9, Training Loss: 0.16074246415138266, Validation Loss: 0.24399356781141252, Validation Accuracy: 0.921875, Validation F1 Score: 0.6666666666666667


100%|██████████| 10/10 [05:06<00:00, 30.65s/it]

Epoch: 10, Training Loss: 0.14537076405303903, Validation Loss: 0.19188100989413215, Validation Accuracy: 0.9375, Validation F1 Score: 0.7058823529411764


## sGNN with Feature Subset

In [20]:
with open("feature_dicts/filtered_features_dict.pkl", "rb") as f:
    feat_dict = pkl.load(f)

In [21]:
all_loader = DataLoader(all_graphs, batch_size=4)

/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [22]:
def add_features(years, graphs, feat_dict, dim):

    zeros = torch.zeros(dim)

    for i in range(len(years)):
        new_x = torch.empty(0, dim)
        year = years[i]

        feat_dict_year = feat_dict[year].combined_features

        for j, country in enumerate(all_nodes):
            if j == 0:
                new_x = torch.stack([zeros])

            elif country in feat_dict_year["country_code"].values:
                tensor_before = graphs[i].x[j]
                country_row = feat_dict_year[feat_dict_year["country_code"] == country]
                country_row = country_row.drop(columns = ["prev_gdp_growth", "country_code", "current_gdp_growth"])
                row_values = country_row.values.tolist()
                row_tensor = torch.tensor(row_values)[0]
                combined_values = torch.cat((tensor_before, row_tensor))

                new_x = torch.cat((new_x, combined_values.unsqueeze(0)), dim=0)

            else:
                new_x = torch.cat((new_x, zeros.unsqueeze(0)), dim=0)

        graphs[i].x = new_x

    return graphs

In [23]:
all_graphs = add_features(years, all_graphs, feat_dict, 59)

In [24]:
all_pairs = get_year_pairs(years)
all_y = check_crisis_years(all_pairs, crisis_years)
all_loader_pairs = get_loader_pairs(all_loader.dataset)

In [25]:
labeled_pairs_all = list(zip(all_loader_pairs, all_y))
flattened_all = [(a, b, c) for ((a, b), c) in labeled_pairs_all]

In [26]:
flattened_train, flattened_test = train_test_split(flattened_all, test_size=0.40, random_state=42)
flattened_test, flattened_val = train_test_split(flattened_test, test_size=0.5, random_state=42)

In [27]:
positive_samples = [item for item in flattened_train if item[2] == 1]
negative_samples = [item for item in flattened_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = positive_samples * (diff // len(positive_samples)) + random.sample(positive_samples, diff % len(positive_samples))
    balanced_data = negative_samples + positive_samples + positive_samples_upsampled
else:
    balanced_data = data

# Shuffle the balanced dataset
random.shuffle(balanced_data)

In [28]:
model = SiameseGNN(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

 10%|█         | 1/10 [00:53<08:04, 53.88s/it]

Epoch: 1, Training Loss: 0.5246067232008549, Validation Loss: 0.3943512145313434, Validation Accuracy: 0.80625, Validation F1 Score: 0.46551724137931033


 20%|██        | 2/10 [01:36<06:18, 47.26s/it]

Epoch: 2, Training Loss: 0.3102763529921105, Validation Loss: 0.2102055678027682, Validation Accuracy: 0.909375, Validation F1 Score: 0.6329113924050633


 30%|███       | 3/10 [06:32<18:45, 160.80s/it]

...

Epoch: 8, Training Loss: 0.1666694018568284, Validation Loss: 0.12153153340041171, Validation Accuracy: 0.9375, Validation F1 Score: 0.7058823529411764


 90%|█████████ | 9/10 [51:43<11:26, 686.46s/it]

Epoch: 9, Training Loss: 0.1505776267802857, Validation Loss: 0.10728849280676514, Validation Accuracy: 0.95, Validation F1 Score: 0.7575757575757577


100%|██████████| 10/10 [1:02:43<00:00, 376.32s/it]

Epoch: 10, Training Loss: 0.13415667740729043, Validation Loss: 0.08108625615695927, Validation Accuracy: 0.959375, Validation F1 Score: 0.7868852459016393


In [23]:
model = SiameseGNN_SAGE(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

 10%|█         | 1/10 [00:30<04:33, 30.44s/it]

Epoch: 1, Training Loss: 0.39061135300011796, Validation Loss: 0.12262269745115191, Validation Accuracy: 0.965625, Validation F1 Score: 0.8533333333333334


 20%|██        | 2/10 [00:57<03:45, 28.23s/it]

Epoch: 2, Training Loss: 0.2324132717426189, Validation Loss: 0.08682704398233909, Validation Accuracy: 0.984375, Validation F1 Score: 0.9275362318840579


 30%|███       | 3/10 [01:24<03:13, 27.69s/it]

...

Epoch: 8, Training Loss: 0.057253993377327624, Validation Loss: 0.07343886076178023, Validation Accuracy: 0.98125, Validation F1 Score: 0.9166666666666667


 90%|█████████ | 9/10 [04:08<00:27, 27.21s/it]

Epoch: 9, Training Loss: 0.0913040674993681, Validation Loss: 0.05216090707835974, Validation Accuracy: 0.984375, Validation F1 Score: 0.9315068493150686


100%|██████████| 10/10 [04:35<00:00, 27.53s/it]

Epoch: 10, Training Loss: 0.057596312563013764, Validation Loss: 0.030969696999818552, Validation Accuracy: 0.9875, Validation F1 Score: 0.9444444444444444


In [29]:
model = SiameseGNN_GIN(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

 10%|█         | 1/10 [00:29<04:23, 29.28s/it]

Epoch: 1, Training Loss: 0.5388779173807243, Validation Loss: 0.427033819607459, Validation Accuracy: 0.728125, Validation F1 Score: 0.2809917355371901


 20%|██        | 2/10 [00:58<03:55, 29.44s/it]

Epoch: 2, Training Loss: 0.3516628711449995, Validation Loss: 0.40659108857798854, Validation Accuracy: 0.74375, Validation F1 Score: 0.3787878787878788


 30%|███       | 3/10 [01:28<03:26, 29.50s/it]

...

Epoch: 8, Training Loss: 0.15329725008401385, Validation Loss: 0.19295876558389863, Validation Accuracy: 0.9375, Validation F1 Score: 0.7222222222222221


 90%|█████████ | 9/10 [04:17<00:28, 28.23s/it]

Epoch: 9, Training Loss: 0.09695610462661736, Validation Loss: 0.10925414055709552, Validation Accuracy: 0.965625, Validation F1 Score: 0.8253968253968254


100%|██████████| 10/10 [04:46<00:00, 28.61s/it]

Epoch: 10, Training Loss: 0.07916570539569057, Validation Loss: 0.17200151187607843, Validation Accuracy: 0.953125, Validation F1 Score: 0.7692307692307692


## Random Feature Subset

In [30]:
with open("feature_dicts/random_features_dict.pkl", "rb") as f:
    feat_dict_random = pkl.load(f)

In [31]:
with open("src/pygcn/all_graphs.pkl", "rb") as f:         
    all_graphs = pkl.load(f)

In [32]:
all_graphs = add_features(years, all_graphs, feat_dict_random, 434)

In [33]:
all_pairs = get_year_pairs(years)
all_y = check_crisis_years(all_pairs, crisis_years)
all_loader_pairs = get_loader_pairs(all_loader.dataset)

In [34]:
labeled_pairs_all = list(zip(all_loader_pairs, all_y))
flattened_all = [(a, b, c) for ((a, b), c) in labeled_pairs_all]

In [35]:
flattened_train, flattened_test = train_test_split(flattened_all, test_size=0.40, random_state=42)
flattened_test, flattened_val = train_test_split(flattened_test, test_size=0.5, random_state=42)

In [36]:
positive_samples = [item for item in flattened_train if item[2] == 1]
negative_samples = [item for item in flattened_train if item[2] == 0]

# Calculate the difference in count
diff = len(negative_samples) - len(positive_samples)

# Upsample positive samples
if diff > 0:
    positive_samples_upsampled = positive_samples * (diff // len(positive_samples)) + random.sample(positive_samples, diff % len(positive_samples))
    balanced_data = negative_samples + positive_samples + positive_samples_upsampled
else:
    balanced_data = data

# Shuffle the balanced dataset
random.shuffle(balanced_data)

In [37]:
model = SiameseGNN(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

 10%|█         | 1/10 [00:37<05:35, 37.31s/it]

Epoch: 1, Training Loss: 0.4994401478328923, Validation Loss: 0.18030089988460532, Validation Accuracy: 0.9, Validation F1 Score: 0.6097560975609756


 20%|██        | 2/10 [01:14<04:58, 37.28s/it]

Epoch: 2, Training Loss: 0.2287323622245777, Validation Loss: 0.19830659957442548, Validation Accuracy: 0.9125, Validation F1 Score: 0.631578947368421


 30%|███       | 3/10 [01:52<04:22, 37.49s/it]

...

Epoch: 8, Training Loss: 0.1478540483564352, Validation Loss: 0.17415061810970656, Validation Accuracy: 0.93125, Validation F1 Score: 0.6944444444444444


 90%|█████████ | 9/10 [05:36<00:37, 37.36s/it]

Epoch: 9, Training Loss: 0.11695554879792472, Validation Loss: 0.1824733758427101, Validation Accuracy: 0.925, Validation F1 Score: 0.6923076923076924


100%|██████████| 10/10 [06:15<00:00, 37.57s/it]

Epoch: 10, Training Loss: 0.13334528002804624, Validation Loss: 0.10680416731320293, Validation Accuracy: 0.95, Validation F1 Score: 0.75


In [38]:
model = SiameseGNN_SAGE(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:32<04:48, 32.06s/it]

Epoch: 1, Training Loss: 0.39902054812754995, Validation Loss: 0.2684114982781466, Validation Accuracy: 0.909375, Validation F1 Score: 0.4912280701754386


 20%|██        | 2/10 [01:00<03:57, 29.73s/it]

Epoch: 2, Training Loss: 0.2679929730612097, Validation Loss: 0.2185995592692052, Validation Accuracy: 0.934375, Validation F1 Score: 0.5714285714285714


...

Epoch: 8, Training Loss: 0.0714946855060778, Validation Loss: 0.07052051206956093, Validation Accuracy: 0.978125, Validation F1 Score: 0.8771929824561403


 90%|█████████ | 9/10 [04:16<00:28, 28.40s/it]

Epoch: 9, Training Loss: 0.05375205374925613, Validation Loss: 0.23170704257099714, Validation Accuracy: 0.9125, Validation F1 Score: 0.631578947368421


100%|██████████| 10/10 [04:50<00:00, 29.02s/it]

Epoch: 10, Training Loss: 0.04655325705862278, Validation Loss: 0.04375610533970757, Validation Accuracy: 0.978125, Validation F1 Score: 0.8627450980392156


In [39]:
model = SiameseGNN_GIN(num_features=balanced_data[0][0].num_node_features)
run_model(model, balanced_data, flattened_val)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:34<05:10, 34.48s/it]

Epoch: 1, Training Loss: 0.4960282989454683, Validation Loss: 0.3852242974913679, Validation Accuracy: 0.7125, Validation F1 Score: 0.3428571428571429


 20%|██        | 2/10 [01:03<04:09, 31.19s/it]

Epoch: 2, Training Loss: 0.3505819629361099, Validation Loss: 0.2887980861123651, Validation Accuracy: 0.846875, Validation F1 Score: 0.505050505050505


...

Epoch: 8, Training Loss: 0.10435694837796874, Validation Loss: 0.10982953479051502, Validation Accuracy: 0.9625, Validation F1 Score: 0.7999999999999999


 90%|█████████ | 9/10 [04:23<00:28, 28.63s/it]

Epoch: 9, Training Loss: 0.16645444669599033, Validation Loss: 0.11311582847702084, Validation Accuracy: 0.946875, Validation F1 Score: 0.7384615384615385


100%|██████████| 10/10 [04:51<00:00, 29.13s/it]

Epoch: 10, Training Loss: 0.09859428378799331, Validation Loss: 0.07590149858406221, Validation Accuracy: 0.975, Validation F1 Score: 0.8571428571428572


## Synthetic Experiments

### Merge
SBM 1: 4 communities

SBM 2: 2 communities

q = 0.2, p = [0.25, 1]